# imports

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import pickle
import qplib as qp
from qplib import log, na, nk, num



# log

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)




log('trace: this is a trace message')
log('debug: this is a debug message')
log('info: this is an info message')
log('warning: this is a warning message')
log('error: this is an error message')

log()

# diff

In [ ]:

import pandas as pd
import numpy as np
import copy
import os
import datetime
import qplib as qp

from IPython.display import display
from ipywidgets import interact, widgets
from pandas.api.extensions import register_dataframe_accessor

from qplib.pandas import _format_df, get_dfs
from qplib.util import log, GREEN, RED, ORANGE, GREEN_LIGHT, RED_LIGHT, ORANGE_LIGHT
from qplib.types import _date, _na, qpDict


df_new, df_old = get_dfs()


print('df_new:')
display(df_new)

print('df_old:')
display(df_old)

print('mode=new:')
display(qp.diff(df_new, df_old, mode='new'))

print('mode=new+:')
display(qp.diff(df_new, df_old, mode='new+'))

print('mode=old:')
display(qp.diff(df_new, df_old, mode='old'))

print('mode=mix:')
display(qp.diff(df_new, df_old, mode='mix'))


In [ ]:

import pandas as pd
import numpy as np
import copy
from qplib.pandas import _format_df
from qplib.util import log, GREEN, ORANGE, RED, GREEN_LIGHT, ORANGE_LIGHT, RED_LIGHT


 

new = 'archive/NET_BM Study_01_export_2024-04-11.xlsx'
old = 'archive/NET_BM Study_01_export_2024-03-15.xlsx'
df1, df2 = qp.get_dfs()


# diff(new, old, mode='mix', returns='test.xlsx')


# tests

In [ ]:

# #run tests in folder "tests" using pytest and create a test report
# !pytest tests --html=tests/test_report.html


In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)


def get_df_simple():
    df = pd.DataFrame({
        'a': [-1, 0, 1],
        'b': [1, 2, 3]
        })
    return df

def get_df_simple_tagged():
    df = pd.DataFrame({
        'meta': ['', '', ''],
        'a': [-1, 0, 1],
        'b': [1, 2, 3]
        })
    df.index = pd.Index([3, 1, 2])
    return df


def get_df():
    df = pd.DataFrame({
        'ID': [10001, 10002, 10003, 20001, 20002, 20003, 30001, 30002, 30003, 30004, 30005],
        'name': ['John Doe', 'Jane Smith', 'Alice Johnson', 'Bob Brown', 'eva white', 'Frank miller', 'Grace TAYLOR', 'Harry Clark', 'IVY GREEN', 'JAck Williams', 'john Doe'],
        'date of birth': ['1995-01-02', '1990/09/14', '1985.08.23', '19800406', '05-11-2007', '06-30-1983', '28-05-1975', '1960Mar08', '1955-Jan-09', '1950 Sep 10', '1945 October 11'],
        'age': [-25, '30', np.nan, None, '40.0', 'forty-five', 'nan', 'unk', '', 'unknown', 35],
        'gender': ['M', 'F', 'Female', 'Male', 'Other', 'm', 'ff', 'NaN', None, 'Mal', 'female'],
        'height': [170, '175.5cm', None, '280', 'NaN', '185', '1', '6ft 1in', -10, '', 200],
        'weight': [70.2, '68', '72.5lb', 'na', '', '75kg', None, '80.3', '130lbs', '82', -65],
        'bp systole': ['20', 130, 'NaN', '140', '135mmhg', '125', 'NAN', '122', '', 130, '45'],
        'bp diastole': [80, '85', 'nan', '90mmHg', np.nan, '75', 'NaN', None, '95', '0', 'NaN'],
        'cholesterol': ['Normal', 'Highe', 'NaN', 'GOOD', 'n.a.', 'High', 'Normal', 'n/a', 'high', '', 'Normal'],
        'diabetes': ['No', 'yes', 'N/A', 'No', 'Y', 'Yes', 'NO', None, 'NaN', 'n', 'Yes'],
        'dose': ['10kg', 'NaN', '15 mg once a day', '20mg', '20 Mg', '25g', 'NaN', None, '30 MG', '35', '40ml']
        })
    return df


def get_df_tagged():
    df1 = get_df()
    df2 = pd.DataFrame('', index=df1.index, columns=['meta', *df1.columns])
    df2.iloc[:, 1:] = df1.loc[:, :]
    return df2


df = qp.get_df()
display(df)

  

param_pairs = [
    (r'name  %%is str;',                 df.loc[:, ['name']]),
    (r'name  %%!is str;',                df.loc[[], ['name']]),
    (r'name  %%is num;',                 df.loc[[], ['name']]),
    (r'name  %%!is num;',                df.loc[:, ['name']]),
    (r'name  %%is na;',                  df.loc[[], ['name']]),
    (r'name  %%!is na;',                 df.loc[:, ['name']]),

    (r'age   %%is int;',                 df.loc[[0,1,4,10], ['age']]),
    (r'age   %%strict is int;',          df.loc[[0,10], ['age']]),
    (r'age   %%is float;',               df.loc[[0,1,2,4,6,10], ['age']]),
    (r'age   %%strict is float;',        df.loc[[2], ['age']]),
    (r'age   %%is na;',                  df.loc[[2,3,6,8], ['age']]),

    (r'weight  %%is int;',               df.loc[[1,9,10], ['weight']]),
    (r'weight  %%strict is int;',        df.loc[[10], ['weight']]),
    (r'weight  %%is float;',             df.loc[[0,1,7,9,10], ['weight']]),
    (r'weight  %%strict is float;',      df.loc[[0], ['weight']]),
    (r'weight  %%is num;',               df.loc[[0,1,4,6,7,9,10], ['weight']]),
    (r'weight  %%strict is num;',        df.loc[[0,10], ['weight']]),
    (r'weight  %%is num;  &&!is na;',    df.loc[[0,1,7,9,10], ['weight']]),

    (r'height       %%is bool;',         df.loc[[6], ['height']]),
    (r'bp diastole  %%is bool;',         df.loc[[9], ['bp diastole']]),
    (r'diabetes     %%is bool;',         df.loc[[0,1,3,4,5,6,9,10], ['diabetes']]),
    (r'diabetes     %%strict is bool;',  df.loc[[], ['diabetes']]),

    (r'diabetes  %%is yn;',              df.loc[[0,1,3,4,5,6,9,10], ['diabetes']]),
    (r'diabetes  %%is na;  //is yn;',    df.loc[:, ['diabetes']]),
    (r'diabetes  %%is yes;',             df.loc[[1,4,5,10], ['diabetes']]),
    (r'diabetes  %%is no;',              df.loc[[0,3,6,9], ['diabetes']]),

    (r'cholesterol  %%is na;',           df.loc[[2,4,7,9], ['cholesterol']]),
    (r'age          %%is na;',           df.loc[[2,3,6,8], ['age']]),
    (r'age          %%strict is na;',           df.loc[[2,3], ['age']]),
    ]
def test(code, expected):
    df = get_df()
    temp = df.q(code)
    result = df.loc[temp.index, temp.columns]
    assert result.equals(expected), qp.diff(result, expected, output='str')

for code, expected in param_pairs:
    test(code, expected)




# typing/pylance

In [ ]:

class A:
    def __init__(self, name: str, val: int) -> None:
        self.name = name
        self.val = val

    def __repr__(self):
        return f'{self.name}: {self.val}'

class B:
    def __init__(self) -> None:
        super().__init__()

    def __setattr__(self, name:str, thing: A):
        super().__setattr__(name, thing)

a: A = A('a', 1)

b = B()

b.test = a

print(b.test)


In [ ]:
class A:
    def __init__(self, name: str, val: int) -> None:
        self.name = name
        self.val = val

    def __repr__(self):
        return f'{self.name}: {self.val}'

class B:
    def __init__(self) -> None:
        super().__init__()
        self._attributes = {}

    def __setattr__(self, name: str, thing: A):
        if name != '_attributes':
            self._attributes[name] = thing
        else:
            super().__setattr__(name, thing)

    def __getattr__(self, name: str):
        return self._attributes.get(name)

a: A = A('a', 1)

b = B()

b.a = a

print(b.a)

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)


VERBOSITY = 3



class Symbol:
    """
    A Symbol used in the query languages syntax.
    """
    def __init__(self, symbol, name, description, **kwargs):
        self.symbol = symbol
        self.name = name
        self.description = description
        for key, value in kwargs.items():
            setattr(self, key, value)

    def details(self):
        return f'name:{self.name}\n\tsymbol:{self.symbol}\n\tdescription{self.description}'

    def __repr__(self):
        return self.name
 
    def __str__(self):
        return self.name


class Symbols:
    """
    Multiple Symbols of the same category are collected in a Symbols object.
    """
    def __init__(self, name, *symbols):
        self.name = name
        self.by_name = {symbol.name: symbol for symbol in symbols}
        self.by_symbol = {symbol.symbol: symbol for symbol in symbols}


    def __getattribute__(self, value):
        if value == 'by_name':
            return super().__getattribute__(value)
        elif value == 'by_symbol':
            return super().__getattribute__(value)
        elif value in self.by_name:
            return self.by_name[value]
        elif value in self.by_symbol:
            return self.by_symbol[value]
        else:
            return super().__getattribute__(value)

    def __setitem__(self, symbol):
        self.by_name[symbol.name] = symbol
        self.by_symbol[symbol.symbol] = symbol
             
    def __getitem__(self, key):
        if key in self.by_symbol:
            return self.by_symbol[key]
        elif key in self.by_name:
            return self.by_name[key]
        else:
            log(f'error: symbol "{key}" not found in "{self.name}"', 'qp.qlang.Symbols.__getitem__', VERBOSITY)
            return None

    def __iter__(self):
        return iter(self.by_name.values())

    def __repr__(self):
        return f'{self.name}:\n\t' + '\n\t'.join([str(val) for key,val in self.by_name.items()])
    
    def __str__(self):
        return self.__repr__()


FLAGS = Symbols('FLAGS',
    Symbol('any', 'ANY', 'select whole row if ANY value in the selected columns fulfills the condition'),
    )

flags_select = set([
    FLAGS.ANY, 
    ])

print(FLAGS)



In [ ]:
from typing import Any
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)

@pd.api.extensions.register_dataframe_accessor('test')
class A:
    def __init__(self, name: str = 'a', val: int = 1) -> None:
        self.name = name
        self.val = val
    
    def __call__(self, text='Hi') -> str:
        return f'{text}, {self.name}'


a = A('a', 1)

df = pd.DataFrame()

df.test('Hi')

df.q()



# qlang

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)
qp.qlang.VERBOSITY = 3

df = qp.get_df()

df1 = pd.DataFrame({
        'meta': ['', '', ''],
        'a': [-1, 0, 1],
        'b': [1, 2, 3]
        })
df1.index = [0, 1, 2]

df.q(
    """
    id   %%each ?1   %%save 1
    id   %%each ?2   %%save 2

    id   %%load 1   &&load 2  $color=red
    """
    )


df.q(
        r"""
        $ verbosity = 3
        $ diff = None

        id /date of birth   %%all?1  $tag1

        is any;
        """
        )

# temp

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
import sys
import shutil
import datetime
import qplib as qp
from qplib import log
pd.set_option('display.max_columns', None)
